In [77]:
import pandas as pd
from bicm import BipartiteGraph
import numpy as np
from tqdm import tqdm
import csv
import itertools 
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, f1_score, classification_report
from sklearn.metrics import roc_curve, roc_auc_score, precision_recall_curve, average_precision_score
from sklearn.metrics import confusion_matrix, f1_score, classification_report
import math

### Functions

In [85]:
def dump_degree_sequences(train,test,fold=0,unseen_folder='VecNet_Unseen_Nodes'):
    
    ligands = list(set(train['InChiKey'].tolist()))
    targets = list(set(train['target_aa_code'].tolist()))
    
    ligands_degree_dict = dict()

    for inchikey_chem in tqdm(ligands):
        sum_df = train[train['InChiKey'] == inchikey_chem]
        ligands_degree_dict[inchikey_chem] = dict()
        ligands_degree_dict[inchikey_chem]['deg_0'] = len(sum_df[sum_df['Y'] == 0])
        ligands_degree_dict[inchikey_chem]['deg_1'] = len(sum_df[sum_df['Y'] == 1])
        
    targets_degree_dict = dict()

    for aa_target in tqdm(targets):
        sum_df = train[train['target_aa_code'] == aa_target]
        targets_degree_dict[aa_target] = dict()
        targets_degree_dict[aa_target]['deg_0'] = len(sum_df[sum_df['Y'] == 0])
        targets_degree_dict[aa_target]['deg_1'] = len(sum_df[sum_df['Y'] == 1])
        
    degree_train_1_0_ligands = [ligands_degree_dict[key_val]['deg_1'] for key_val in tqdm(ligands_degree_dict.keys())]
    degree_train_0_1_ligands = [ligands_degree_dict[key_val]['deg_0'] for key_val in tqdm(ligands_degree_dict.keys())]

    degree_train_1_0_targets = [targets_degree_dict[key_val]['deg_1'] for key_val in tqdm(targets_degree_dict.keys())]
    degree_train_0_1_targets = [targets_degree_dict[key_val]['deg_0'] for key_val in tqdm(targets_degree_dict.keys())]
    
    with open('../data/sars-busters-consolidated/GitData/' + unseen_folder + '/Degree_Sequences/degreetrain10ligands_' + str(fold) + '.txt', 'w') as file:
        for degree in degree_train_1_0_ligands:
            file.write("%i\n" % degree)
        
    file.close()
    
    with open('../data/sars-busters-consolidated/GitData/' + unseen_folder + '/Degree_Sequences/degreetrain01ligands_' + str(fold) + '.txt', 'w') as file:
        for degree in degree_train_0_1_ligands:
            file.write("%i\n" % degree)
        
    file.close()
    
    with open('../data/sars-busters-consolidated/GitData/' + unseen_folder + '/Degree_Sequences/degreetrain10targets_' + str(fold) + '.txt', 'w') as file:
        for degree in degree_train_1_0_targets:
            file.write("%i\n" % degree)
        
    file.close()
    
    with open('../data/sars-busters-consolidated/GitData/' + unseen_folder + '/Degree_Sequences/degreetrain01targets_' + str(fold) + '.txt', 'w') as file:
        for degree in degree_train_0_1_targets:
            file.write("%i\n" % degree)
        
    file.close()
    
    textfile = open('../data/sars-busters-consolidated/GitData/' + unseen_folder + '/Degree_Sequences/ligands_' + str(fold) + '.txt', "w")
    for element in ligands:
        textfile.write(element + "\n")
    textfile.close()
    
    textfile = open('../data/sars-busters-consolidated/GitData/' + unseen_folder + '/Degree_Sequences/targets_' + str(fold) + '.txt', "w")
    for element in targets:
        textfile.write(element + "\n")
    textfile.close()
    
    return 

In [81]:
def get_configuration_model_performance(train,test,ligand_file_path,target_file_path,summat10_file_path,summat01_file_path):
    
    text_file = open(ligand_file_path, "r") # Rows of the adjacency matrix in order
    ligands = text_file.readlines()

    text_file = open(target_file_path, "r") # Columns of the adjacency matrix in order 
    targets = text_file.readlines()
    
    ligands = [j.replace('\n','') for j in tqdm(ligands)]
    targets = [j.replace('\n','') for j in tqdm(targets)]
    
    number_ligands = len(ligands)
    number_targets = len(targets)
    
    train_pos = train[train['Y'] == 1]
    train_neg = train[train['Y'] == 0]
    
    pos_deg_0_ligands = []
    pos_deg_0_targets = []
    neg_deg_0_ligands = []
    neg_deg_0_targets = []
    
    ligand_degree_ratio = dict()
    ligand_all_average = []

    for ligand in tqdm(ligands):
        pos_deg = len(train_pos[train_pos['InChiKey'] == ligand])
        neg_deg = len(train_neg[train_neg['InChiKey'] == ligand])
        ligand_degree_ratio[ligand] = dict()
        ligand_degree_ratio[ligand]['deg_ratio'] = pos_deg / (pos_deg + neg_deg)
        ligand_degree_ratio[ligand]['deg_avg'] = pos_deg / number_targets 
        ligand_all_average.append(pos_deg / number_targets)
        if pos_deg == 0:
            pos_deg_0_ligands.append(ligand)
        if neg_deg == 0:
            neg_deg_0_ligands.append(ligand)
    
    ligands_all_avg = sum(ligand_all_average) / number_ligands
    
    targets_degree_ratio = dict()
    target_all_average = []

    for target in tqdm(targets):
        pos_deg = len(train_pos[train_pos['target_aa_code'] == target])
        neg_deg = len(train_neg[train_neg['target_aa_code'] == target])
        targets_degree_ratio[target] = dict()
        targets_degree_ratio[target]['deg_ratio'] = pos_deg / (pos_deg + neg_deg)
        targets_degree_ratio[target]['deg_avg'] = pos_deg / number_ligands
        target_all_average.append(pos_deg / number_ligands)
        if pos_deg == 0:
            pos_deg_0_targets.append(target)
        if neg_deg == 0:
            neg_deg_0_targets.append(target)
    
    targets_all_avg = sum(target_all_average) / number_targets
    
    print('Ligands with positive degree 0: ',len(pos_deg_0_ligands))
    print('Ligands with negative degree 0: ',len(neg_deg_0_ligands))
    print('Targets with positive degree 0: ',len(pos_deg_0_targets))
    print('Targets with negative degree 0: ',len(neg_deg_0_targets))
    
    
    pos_annotated_ligands = list(set(ligands)-set(pos_deg_0_ligands))
    pos_annotated_targets = list(set(targets)-set(pos_deg_0_targets))
    
    neg_annotated_ligands = list(set(ligands)-set(neg_deg_0_ligands))
    neg_annotated_targets = list(set(targets)-set(neg_deg_0_targets))
    
    summat10 = np.loadtxt(open(summat10_file_path, "rb"), delimiter=",", skiprows=0) # Output of MATLAB run 
    summat01 = np.loadtxt(open(summat01_file_path, "rb"), delimiter=",", skiprows=0) # Output of MATLAB run 
    
    test_probabilty_predicted_conditioned = []

    ## Average conditional probability
    conditoned_summat = np.divide(summat10,np.add(summat10,summat01)) # Elementwise pos_deg / (pos_deg + neg_deg)
    conditoned_summat = conditoned_summat[~np.isnan(conditoned_summat)] 
    average_conditional_probability = sum(conditoned_summat) / len(conditoned_summat) # Average over valid conditional probabilities

    drop_nan = []
    
    for index, row in tqdm(test.iterrows()):      
    
        if row['InChiKey'] in pos_annotated_ligands and row['target_aa_code'] in pos_annotated_targets:
            p10 = summat10[ligands.index(row['InChiKey']),targets.index(row['target_aa_code'])]
            p01 = summat01[ligands.index(row['InChiKey']),targets.index(row['target_aa_code'])]
            p10_conditioned = p10 / (p10 + p01)
        
        elif row['InChiKey'] in pos_annotated_ligands and row['target_aa_code'] not in pos_annotated_targets:
            p10_conditioned = ligand_degree_ratio[row['InChiKey']]['deg_ratio']  ## k_+ / (k_+ + k_-)
        
        elif row['InChiKey'] not in pos_annotated_ligands and row['target_aa_code'] in pos_annotated_targets:
            p10_conditioned = targets_degree_ratio[row['target_aa_code']]['deg_ratio'] ## k_+ / (k_+ + k_-)

        else:
            p10_conditioned = average_conditional_probability
            
        if math.isnan(p10_conditioned): 
            drop_nan.append(index)
        else:
            test_probabilty_predicted_conditioned.append(p10_conditioned)
            
        ## Performance on the test dataset

    print('AUC: ', roc_auc_score(test.drop(drop_nan)['Y'].tolist(), test_probabilty_predicted_conditioned))
    print('AUP: ', average_precision_score(test.drop(drop_nan)['Y'].tolist(), test_probabilty_predicted_conditioned))
        
    return roc_auc_score(test.drop(drop_nan)['Y'].tolist(), test_probabilty_predicted_conditioned), average_precision_score(test.drop(drop_nan)['Y'].tolist(), test_probabilty_predicted_conditioned)

### Generated the degree files - Unseen Nodes and Edges

In [19]:
for fold in tqdm(range(5)):

    train = pd.read_csv('../data/sars-busters-consolidated/GitData/VecNet_Unseen_Nodes/train_' + str(fold) + '.csv')
    edges_test = pd.read_csv('../data/sars-busters-consolidated/GitData/VecNet_Unseen_Nodes/test_unseen_edges_' + str(fold) + '.csv')
    nodes_test = pd.read_csv('../data/sars-busters-consolidated/GitData/VecNet_Unseen_Nodes/test_unseen_nodes_' + str(fold) + '.csv')
    dump_degree_sequences(train,nodes_test,fold=fold,unseen_folder='VecNet_Unseen_Nodes')

100%|██████████| 2705/2705 [00:08<00:00, 316.16it/s]

100%|██████████| 3049/3049 [00:09<00:00, 320.25it/s]

100%|██████████| 2705/2705 [00:00<00:00, 1270218.58it/s]

100%|██████████| 2705/2705 [00:00<00:00, 1326349.35it/s]

100%|██████████| 3049/3049 [00:00<00:00, 1321665.24it/s]

100%|██████████| 2525/2525 [00:07<00:00, 326.04it/s]

100%|██████████| 2994/2994 [00:08<00:00, 333.85it/s]

100%|██████████| 2525/2525 [00:00<00:00, 1070516.28it/s]

100%|██████████| 2525/2525 [00:00<00:00, 1151654.81it/s]

100%|██████████| 2994/2994 [00:00<00:00, 1369437.97it/s]

100%|██████████| 3394/3394 [00:11<00:00, 283.67it/s]

100%|██████████| 3184/3184 [00:10<00:00, 299.84it/s]

100%|██████████| 3394/3394 [00:00<00:00, 1237006.24it/s]

100%|██████████| 3394/3394 [00:00<00:00, 1246756.68it/s]

100%|██████████| 3184/3184 [00:00<00:00, 1285585.67it/s]

100%|██████████| 2271/2271 [00:06<00:00, 344.63it/s]

100%|██████████| 2965/2965 [00:08<00:00, 348.61it/s]

100%|██████████| 2271/2271 [00:00<00:00, 11890

######## Now run the MATLAB code in Configuration Model - 5 fold folder to generate the matrices. #########

### Get Unseen Nodes and Edges Test Performance

In [83]:
auc_nodes = []
aup_nodes = []
auc_edges =[]
aup_edges = []


for fold in tqdm(range(5)):

    train = pd.read_csv('../data/sars-busters-consolidated/GitData/VecNet_Unseen_Nodes/train_' + str(fold) + '.csv')
    edges_test = pd.read_csv('../data/sars-busters-consolidated/GitData/VecNet_Unseen_Nodes/test_unseen_edges_' + str(fold) + '.csv')
    nodes_test = pd.read_csv('../data/sars-busters-consolidated/GitData/VecNet_Unseen_Nodes/test_unseen_nodes_' + str(fold) + '.csv')

    ligand_file_path = '/data/sars-busters-consolidated/GitData/VecNet_Unseen_Nodes/Degree_Sequences/ligands_' + str(fold) + '.txt'
    target_file_path = '/data/sars-busters-consolidated/GitData/VecNet_Unseen_Nodes/Degree_Sequences/targets_' + str(fold) + '.txt'
    summat10_file_path = '/data/sars-busters-consolidated/GitData/VecNet_Unseen_Nodes/Degree_Sequences/summat10_' + str(fold) + '.csv'
    summat01_file_path = '/data/sars-busters-consolidated/GitData/VecNet_Unseen_Nodes/Degree_Sequences/summat01_' + str(fold) + '.csv'

    auc, aup = get_configuration_model_performance(train,nodes_test,ligand_file_path,target_file_path,summat10_file_path,summat01_file_path)
    auc_nodes.append(auc)
    aup_nodes.append(aup)
    
    auc, aup = get_configuration_model_performance(train,edges_test,ligand_file_path,target_file_path,summat10_file_path,summat01_file_path)
    auc_edges.append(auc)
    aup_edges.append(aup)

100%|██████████| 2705/2705 [00:00<00:00, 1235634.10it/s]

100%|██████████| 3049/3049 [00:00<00:00, 786738.41it/s]

100%|██████████| 2705/2705 [00:07<00:00, 374.91it/s]

100%|██████████| 3049/3049 [00:07<00:00, 400.68it/s]


Ligands with positive degree 0:  446
Ligands with negative degree 0:  69
Targets with positive degree 0:  2069
Targets with negative degree 0:  334


/miniconda/lib/python3.6/site-packages/ipykernel_launcher.py:75: RuntimeWarning: invalid value encountered in true_divide

0it [00:00, ?it/s]
280it [00:00, 2796.12it/s]
562it [00:00, 2809.98it/s]
854it [00:00, 2856.76it/s]
1146it [00:00, 2881.26it/s]
1437it [00:00, 2890.89it/s]
1727it [00:00, 2891.77it/s]
2017it [00:00, 2880.09it/s]
2306it [00:00, 2844.12it/s]
2591it [00:00, 2736.70it/s]
2878it [00:01, 2775.02it/s]
3166it [00:01, 2805.07it/s]
3456it [00:01, 2831.00it/s]
3746it [00:01, 2851.22it/s]
4040it [00:01, 2875.38it/s]
4330it [00:01, 2879.90it/s]
4619it [00:01, 2881.12it/s]
4910it [00:01, 2888.36it/s]
5200it [00:01, 2891.23it/s]
5490it [00:01, 2893.14it/s]
5780it [00:02, 2892.77it/s]
6070it [00:02, 2850.30it/s]
6491it [00:02, 2846.63it/s]

100%|██████████| 2705/2705 [00:00<00:00, 801016.12it/s]

100%|██████████| 3049/3049 [00:00<00:00, 508445.96it/s]

  1%|▏         | 39/2705 [00:00<00:06, 385.95it/s]


AUC:  0.5
AUP:  0.482976428901556


100%|██████████| 2705/2705 [00:07<00:00, 386.09it/s]

100%|██████████| 3049/3049 [00:07<00:00, 400.47it/s]


...

AUC:  0.5
AUP:  0.4646845759130697


100%|██████████| 3441/3441 [00:10<00:00, 335.08it/s]

100%|██████████| 3236/3236 [00:09<00:00, 358.29it/s]


Ligands with positive degree 0:  797
Ligands with negative degree 0:  83
Targets with positive degree 0:  2065
Targets with negative degree 0:  239


/miniconda/lib/python3.6/site-packages/ipykernel_launcher.py:75: RuntimeWarning: invalid value encountered in true_divide

0it [00:00, ?it/s]
267it [00:00, 2660.18it/s]
534it [00:00, 2640.26it/s]
799it [00:00, 2615.47it/s]
1061it [00:00, 2603.51it/s]
1333it [00:00, 2641.99it/s]
1602it [00:00, 2654.48it/s]
1868it [00:00, 2654.75it/s]
2136it [00:00, 2661.57it/s]
2404it [00:00, 2666.03it/s]
2671it [00:01, 2647.45it/s]
2939it [00:01, 2655.47it/s]
3209it [00:01, 2666.17it/s]
3696it [00:01, 2655.78it/s]
100%|██████████| 5/5 [05:32<00:00, 66.54s/it]

AUC:  0.7441010354978903
AUP:  0.7552506710842388


In [84]:
print('Unseen Nodes')
print('AUC: ', np.mean(auc_nodes), '+-', np.std(auc_nodes))
print('AUP: ', np.mean(aup_nodes), '+-', np.std(aup_nodes))

print('Unseen Edges')
print('AUC: ', np.mean(auc_edges), '+-', np.std(auc_edges))
print('AUP: ', np.mean(aup_edges), '+-', np.std(aup_edges))

Unseen Nodes
AUC:  0.5 +- 0.0
AUP:  0.4694108013484294 +- 0.014157461223848512
Unseen Edges
AUC:  0.7225003718211122 +- 0.015038589509461478
AUP:  0.7365911246206653 +- 0.016380999195983965


### Generated the degree files - Unseen Targets

In [87]:
for fold in tqdm(range(5)):

    train = pd.read_csv('../data/sars-busters-consolidated/GitData/VecNet_Unseen_Targets/train_' + str(fold) + '.csv')
    edges_test = pd.read_csv('../data/sars-busters-consolidated/GitData/VecNet_Unseen_Targets/test_unseen_edges_' + str(fold) + '.csv')
    nodes_test = pd.read_csv('../data/sars-busters-consolidated/GitData/VecNet_Unseen_Targets/test_unseen_nodes_' + str(fold) + '.csv')
    dump_degree_sequences(train,nodes_test,fold=fold,unseen_folder='VecNet_Unseen_Targets')

100%|██████████| 6751/6751 [00:40<00:00, 164.89it/s]

100%|██████████| 3594/3594 [00:19<00:00, 183.33it/s]

100%|██████████| 6751/6751 [00:00<00:00, 1206619.78it/s]

100%|██████████| 6751/6751 [00:00<00:00, 1324713.28it/s]

100%|██████████| 3594/3594 [00:00<00:00, 1188920.94it/s]

100%|██████████| 6765/6765 [00:39<00:00, 170.49it/s]

100%|██████████| 3533/3533 [00:19<00:00, 185.24it/s]

100%|██████████| 6765/6765 [00:00<00:00, 1227110.09it/s]

100%|██████████| 6765/6765 [00:00<00:00, 1350650.54it/s]

100%|██████████| 3533/3533 [00:00<00:00, 1335479.09it/s]

100%|██████████| 6755/6755 [00:39<00:00, 170.48it/s]

100%|██████████| 3547/3547 [00:18<00:00, 187.09it/s]

100%|██████████| 6755/6755 [00:00<00:00, 868962.54it/s]

100%|██████████| 6755/6755 [00:00<00:00, 835743.00it/s]

100%|██████████| 3547/3547 [00:00<00:00, 870367.77it/s]

100%|██████████| 6753/6753 [00:39<00:00, 169.25it/s]

100%|██████████| 3527/3527 [00:18<00:00, 187.07it/s]

100%|██████████| 6753/6753 [00:00<00:00, 1167475.

### Get Unseen Targets Test Performance

In [88]:
auc_targets = []
aup_targets = []

for fold in tqdm(range(5)):

    train = pd.read_csv('../data/sars-busters-consolidated/GitData/VecNet_Unseen_Targets/train_' + str(fold) + '.csv')
    edges_test = pd.read_csv('../data/sars-busters-consolidated/GitData/VecNet_Unseen_Targets/test_unseen_edges_' + str(fold) + '.csv')
    nodes_test = pd.read_csv('../data/sars-busters-consolidated/GitData/VecNet_Unseen_Targets/test_unseen_nodes_' + str(fold) + '.csv')

    ligand_file_path = '/data/sars-busters-consolidated/GitData/VecNet_Unseen_Targets/Degree_Sequences/ligands_' + str(fold) + '.txt'
    target_file_path = '/data/sars-busters-consolidated/GitData/VecNet_Unseen_Targets/Degree_Sequences/targets_' + str(fold) + '.txt'
    summat10_file_path = '/data/sars-busters-consolidated/GitData/VecNet_Unseen_Targets/Degree_Sequences/summat10_' + str(fold) + '.csv'
    summat01_file_path = '/data/sars-busters-consolidated/GitData/VecNet_Unseen_Targets/Degree_Sequences/summat01_' + str(fold) + '.csv'

    auc, aup = get_configuration_model_performance(train,nodes_test,ligand_file_path,target_file_path,summat10_file_path,summat01_file_path)
    auc_targets.append(auc)
    aup_targets.append(aup)

100%|██████████| 6751/6751 [00:00<00:00, 1213029.44it/s]

100%|██████████| 3594/3594 [00:00<00:00, 858789.30it/s]

100%|██████████| 6751/6751 [00:36<00:00, 185.03it/s]

100%|██████████| 3594/3594 [00:17<00:00, 205.00it/s]


Ligands with positive degree 0:  1487
Ligands with negative degree 0:  219
Targets with positive degree 0:  621
Targets with negative degree 0:  0


/miniconda/lib/python3.6/site-packages/ipykernel_launcher.py:75: RuntimeWarning: invalid value encountered in true_divide

0it [00:00, ?it/s]
199it [00:00, 1988.80it/s]
398it [00:00, 1961.65it/s]
596it [00:00, 1967.92it/s]
795it [00:00, 1973.25it/s]
995it [00:00, 1979.14it/s]
1195it [00:00, 1983.94it/s]
1394it [00:00, 1973.85it/s]
1592it [00:00, 1970.39it/s]
1797it [00:00, 1991.74it/s]
1998it [00:01, 1996.52it/s]
2198it [00:01, 1978.12it/s]
2402it [00:01, 1995.51it/s]
2603it [00:01, 1998.72it/s]
2803it [00:01, 1992.33it/s]
3011it [00:01, 2016.03it/s]
3213it [00:01, 2006.41it/s]
3414it [00:01, 2005.61it/s]
3617it [00:01, 2009.87it/s]
3818it [00:01, 1985.95it/s]
4024it [00:02, 2005.61it/s]
4225it [00:02, 1998.69it/s]
4432it [00:02, 2017.42it/s]
4634it [00:02, 2009.19it/s]
4838it [00:02, 2017.93it/s]
5040it [00:02, 2004.51it/s]
5241it [00:02, 1996.58it/s]
5441it [00:02, 1991.05it/s]
5643it [00:02, 1997.00it/s]
5845it [00:02, 2002.02it/s]
6046it [00:03, 1986.40it/s]
6247it [00:03, 1992.84i

AUC:  0.7561647780736922
AUP:  0.731495556420381




100%|██████████| 6765/6765 [00:35<00:00, 189.18it/s]

100%|██████████| 3533/3533 [00:17<00:00, 207.64it/s]


...

AUC:  0.7388026451315681
AUP:  0.7019190925176266




100%|██████████| 6753/6753 [00:35<00:00, 189.88it/s]

100%|██████████| 3527/3527 [00:17<00:00, 205.52it/s]


Ligands with positive degree 0:  1796
Ligands with negative degree 0:  200
Targets with positive degree 0:  613
Targets with negative degree 0:  0


 60%|██████    | 3/5 [05:53<03:55, 117.93s/it]


ValueError: Wrong number of columns at line 357

In [89]:
print('Unseen Targets')
print('AUC: ', np.mean(auc_targets), '+-', np.std(auc_targets))
print('AUP: ', np.mean(aup_targets), '+-', np.std(aup_targets))

Unseen Targets
AUC:  0.7388888494838263 +- 0.014070675880443466
AUP:  0.7056593953880822 +- 0.019746088476437614
